In [85]:
class colors:
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    RED = "\033[31m"
    GREEN = "\033[32m"
    GRAY = "\033[37m"


print(f"{colors.BOLD}{colors.RED}test")

test


In [112]:
empty_board: list[list[int]] = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
]


def print_board(
    board: list[list[int]],
    highlights: list[tuple[int, int]] | None = None,
    color_code: str | None = None,
) -> None:
    print(f"{colors.GRAY}┏━━━┯━━━┯━━━┳━━━┯━━━┯━━━┳━━━┯━━━┯━━━┓")
    for i in range(len(board)):
        row: list[str] = [str(item) for item in board[i]]
        for j in range(len(row)):
            if row[j] == "0":
                row[j] = "x"
            if highlights is not None:
                if (i, j) in highlights:
                    row[j] = f"{color_code}{row[j]}{colors.ENDC}{colors.GRAY}"
            else:
                row[j] = f"{colors.ENDC}{row[j]}{colors.GRAY}"

        print(
            f"┃ {row[0]} │ {row[1]} │ {row[2]} ┃ {row[3]} │ {row[4]} │ {row[5]} ┃ {row[6]} │ {row[7]} │ {row[8]} ┃"
        )
        if i % 3 == 2 and i != 8:
            print("┣━━━┿━━━┿━━━╋━━━┿━━━┿━━━╋━━━┿━━━┿━━━┫")
        elif i != 8:
            print("┠───┼───┼───╂───┼───┼───╂───┼───┼───┨")
    print("┗━━━┷━━━┷━━━┻━━━┷━━━┷━━━┻━━━┷━━━┷━━━┛")


print_board(empty_board, [(1, 2), (3, 4)], colors.GREEN + colors.BOLD)

┏━━━┯━━━┯━━━┳━━━┯━━━┯━━━┳━━━┯━━━┯━━━┓
┃ x │ x │ x ┃ x │ x │ x ┃ x │ x │ x ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ x │ x │ x ┃ x │ x │ x ┃ x │ x │ x ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ x │ x │ x ┃ x │ x │ x ┃ x │ x │ x ┃
┣━━━┿━━━┿━━━╋━━━┿━━━┿━━━╋━━━┿━━━┿━━━┫
┃ x │ x │ x ┃ x │ x │ x ┃ x │ x │ x ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ x │ x │ x ┃ x │ x │ x ┃ x │ x │ x ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ x │ x │ x ┃ x │ x │ x ┃ x │ x │ x ┃
┣━━━┿━━━┿━━━╋━━━┿━━━┿━━━╋━━━┿━━━┿━━━┫
┃ x │ x │ x ┃ x │ x │ x ┃ x │ x │ x ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ x │ x │ x ┃ x │ x │ x ┃ x │ x │ x ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ x │ x │ x ┃ x │ x │ x ┃ x │ x │ x ┃
┗━━━┷━━━┷━━━┻━━━┷━━━┷━━━┻━━━┷━━━┷━━━┛


In [121]:
# Solver

import copy


starting_board: list[list[int]] = [
    [0, 0, 0, 0, 0, 0, 0, 3, 0],
    [0, 0, 8, 0, 2, 0, 1, 6, 0],
    [5, 0, 0, 3, 0, 0, 7, 0, 0],
    [1, 0, 0, 0, 0, 4, 0, 9, 0],
    [0, 0, 0, 7, 6, 1, 0, 0, 0],
    [0, 7, 0, 2, 0, 0, 0, 0, 1],
    [0, 0, 7, 0, 0, 5, 0, 0, 2],
    [0, 2, 9, 0, 8, 0, 5, 0, 0],
    [0, 4, 0, 0, 0, 0, 0, 0, 0],
]


def is_valid(board: list[list[int]], row: int, col: int, num: int) -> bool:
    for x in range(9):
        if board[row][x] == num or board[x][col] == num:
            return False
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False
    return True


def _solve_brute_force(board: list[list[int]]) -> list[list[int]] | None:
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                for num in range(1, 10):
                    if is_valid(board, row, col, num):
                        board[row][col] = num
                        if _solve_brute_force(board):
                            return board
                        board[row][col] = 0
                return None
    return board


def solve_brute_force(board: list[list[int]]) -> list[list[int]]:
    board_copy: list[list[int]] = copy.deepcopy(board)
    result: list[list[int]] | None = _solve_brute_force(board_copy)
    if result is None:
        raise ValueError("No solution exists for the given Sudoku puzzle.")
    return result


# Extract positions of given numbers
given_positions: list[tuple[int, int]] = []
for row in range(9):
    for col in range(9):
        if starting_board[row][col] != 0:
            given_positions.append((row, col))
try:
    print_board(
        solve_brute_force(starting_board),
        given_positions,
        color_code=colors.BOLD + colors.ENDC,
    )
except ValueError as e:
    print(e)

┏━━━┯━━━┯━━━┳━━━┯━━━┯━━━┳━━━┯━━━┯━━━┓
┃ 2 │ 6 │ 1 ┃ 8 │ 5 │ 7 ┃ 4 │ 3 │ 9 ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ 7 │ 3 │ 8 ┃ 4 │ 2 │ 9 ┃ 1 │ 6 │ 5 ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ 5 │ 9 │ 4 ┃ 3 │ 1 │ 6 ┃ 7 │ 2 │ 8 ┃
┣━━━┿━━━┿━━━╋━━━┿━━━┿━━━╋━━━┿━━━┿━━━┫
┃ 1 │ 8 │ 6 ┃ 5 │ 3 │ 4 ┃ 2 │ 9 │ 7 ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ 9 │ 5 │ 2 ┃ 7 │ 6 │ 1 ┃ 8 │ 4 │ 3 ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ 4 │ 7 │ 3 ┃ 2 │ 9 │ 8 ┃ 6 │ 5 │ 1 ┃
┣━━━┿━━━┿━━━╋━━━┿━━━┿━━━╋━━━┿━━━┿━━━┫
┃ 3 │ 1 │ 7 ┃ 6 │ 4 │ 5 ┃ 9 │ 8 │ 2 ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ 6 │ 2 │ 9 ┃ 1 │ 8 │ 3 ┃ 5 │ 7 │ 4 ┃
┠───┼───┼───╂───┼───┼───╂───┼───┼───┨
┃ 8 │ 4 │ 5 ┃ 9 │ 7 │ 2 ┃ 3 │ 1 │ 6 ┃
┗━━━┷━━━┷━━━┻━━━┷━━━┷━━━┻━━━┷━━━┷━━━┛
